In [1]:
!git clone https://github.com/lucas-battisti/custom-pytorch-lightning-module.git
!mv "custom-pytorch-lightning-module/modules.py" "modules.py"
!rm -rf "custom-pytorch-lightning-module"



!git clone https://github.com/lucas-battisti/pytorch-blocks.git
!mv "pytorch-blocks/blocks.py" "blocks.py"
!rm -rf "pytorch-blocks"

Cloning into 'custom-pytorch-lightning-module'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 46 (delta 25), reused 28 (delta 13), pack-reused 0
Receiving objects: 100% (46/46), 22.47 KiB | 1.32 MiB/s, done.
Resolving deltas: 100% (25/25), done.
Cloning into 'pytorch-blocks'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 29 (delta 15), reused 18 (delta 8), pack-reused 0
Receiving objects: 100% (29/29), 20.19 KiB | 1.19 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [2]:
import dt
import pandas as pd

In [10]:
df

,RA,DEC,MU_MAX_INST,FWHM_n,PhotoFlagDet,u_PStotal,e_u_PStotal,J0378_PStotal,e_J0378_PStotal,J0395_PStotal,...,nDet_PStotal,W1_MAG,W2_MAG,e_W1_MAG,e_W2_MAG,FUVmag,NUVmag,e_FUVmag,e_NUVmag,Z
ID,,,,,,,,,,,,,,,,,,,,,
iDR5_3_0_STRIPE82-0074_0030854,50.950031,0.691337,19.122868,1.744728,0.0,21.855314,0.224955,21.838514,0.311987,21.972036,...,12.0,19.123713,19.291985,0.033687,0.083541,NaN,NaN,NaN,NaN,0.247318
iDR5_3_SPLUS-n02s48_0031444,214.704224,-1.127490,18.499992,2.070420,0.0,21.384974,0.452150,21.922901,0.671358,20.816227,...,12.0,18.894621,19.056978,0.025455,0.062376,NaN,NaN,NaN,NaN,0.181604
iDR5_3_0_STRIPE82-0023_0033419,15.010666,-0.082431,20.077642,1.923943,0.0,NaN,NaN,NaN,NaN,21.636702,...,10.0,18.399147,19.160164,0.019780,0.081629,NaN,NaN,NaN,NaN,0.639951
iDR5_3_SPLUS-n01s22_0038877,179.694824,0.403215,18.557959,1.640066,0.0,23.573034,2.673750,22.158922,0.911359,21.302311,...,12.0,18.828346,18.827257,0.027340,0.058215,NaN,NaN,NaN,NaN,0.226022
iDR5_3_0_STRIPE82-0169_0034291,358.984741,-0.114401,18.403070,1.551156,0.0,21.509279,0.727081,20.962872,0.447041,21.956846,...,12.0,18.533161,18.838491,0.020226,0.054294,NaN,NaN,NaN,NaN,0.137109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
iDR5_3_0_STRIPE82_0128_0032437,328.602417,0.698312,18.618523,5.217855,3.0,22.137739,21.924606,21.631830,22.472164,21.187452,...,12.0,17.693741,18.143261,0.010276,0.030319,NaN,NaN,NaN,NaN,0.209080
iDR5_3_0_STRIPE82-0120_0012890,323.410156,0.363115,17.319054,2.877129,0.0,22.417023,1.007946,21.958624,0.881726,NaN,...,11.0,17.536827,17.679157,0.009996,0.021091,NaN,NaN,NaN,NaN,0.371750
iDR5_3_SPLUS-s35s31_0028721,57.084660,-45.228428,20.006327,1.954982,0.0,NaN,NaN,24.126984,6.650383,NaN,...,9.0,20.031857,20.208897,0.054559,0.129773,NaN,NaN,NaN,NaN,0.354642


In [20]:

df = pd.read_csv('data/galax.csv').drop('ID', axis=1).astype('float32')


In [21]:
set_size = [0.6, 0.2, 0.2]

df = pd.read_csv('data/galax.csv').drop('ID', axis=1).astype('float32')

dtfr = dt.frame(df = df,
         set_size=set_size)

f = dtfr.features(['g', 'j','u'])['complete']
e_f = dtfr.features(['e_g', 'e_j','e_u'])['complete']

z = dtfr.features(['z'])['complete']

In [25]:
import lightning as L

from modules import RegressionModule as Reg
from blocks import LinearBlock, Conv1dBlock, Conv2dBlock
from data import FF_Dataset, Custom_DataModule, CNN1D_Dataset, CNN2D_Dataset

import torch
from torch import nn
from torch.nn import MaxPool1d, AvgPool1d
from torch.optim.lr_scheduler import ReduceLROnPlateau

from math import floor

In [26]:
dm = {}
m = {}

# FF with filters and no errors

In [27]:
str = 'ff_filters'

dm[str] = Custom_DataModule(xez = (f, None, z),
                       dataset_class=FF_Dataset,
                       norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

l1 = LinearBlock(in_features=16, out_features=12,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 12},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=12, out_features=12,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 12},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=12, out_features=1)

sequential = nn.Sequential(l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2024-05-08 08:01:13.545457: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-08 08:01:14.196437: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 08:01:15.631421: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 421   
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
421     

# FF with filters and errors

In [ ]:
str = 'ff_filters_errors'

dm[str] = Custom_DataModule(xez = (f, e_f, z),
                       dataset_class=FF_Dataset,
                       norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

l1 = LinearBlock(in_features=32, out_features=24,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 24},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=24, out_features=24,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 24},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=24, out_features=1)

sequential = nn.Sequential(l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 1.5 K 
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
1.5 K     Trainable params
0         Non-trainable params
1.5 K     Total params
0.006     Total estimated model params size (MB)
INFO:lightning.

[{}]

# FF with diffs

In [ ]:
str = 'ff_diffs'

dm[str] = Custom_DataModule(xez = (d, None, z),
                       dataset_class=FF_Dataset,
                       norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

l1 = LinearBlock(in_features=15, out_features=12,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 12},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=12, out_features=12,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 12},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=12, out_features=1)

sequential = nn.Sequential(l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 409   
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
409       Trainable params
0         Non-trainable params
409       Total params
0.002     Total estimated model params size (MB)
INFO:lightning.

[{}]

# FF with diffs and errors

In [ ]:
str = 'ff_diffs_errors'

dm[str] = Custom_DataModule(xez = (d, e_d, z),
                       dataset_class=FF_Dataset,
                       norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

l1 = LinearBlock(in_features=30, out_features=24,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 24},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=24, out_features=24,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 24},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=24, out_features=1)

sequential = nn.Sequential(l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 1.5 K 
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
1.5 K     Trainable params
0         Non-trainable params
1.5 K     Total params
0.006     Total estimated model params size (MB)
INFO:lightning.

[{}]

# CNN1D_no with filters

In [ ]:
str = 'cnn1d_no_filters'

dm[str] = Custom_DataModule(xez = (f, e_f, z),
                       dataset_class=CNN1D_Dataset,
                       version='no', norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

c1 = Conv1dBlock(in_channels=1, out_channels=32,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv1dBlock(in_channels=32, out_channels=64,
                 kernel_size=2,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

flatten = nn.Flatten()

l1 = LinearBlock(in_features=192, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=144, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=144, out_features=1)

sequential = nn.Sequential(c1, c2, flatten,
                           l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 53.9 K
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
53.9 K    Trainable params
0         Non-trainable params
53.9 K    Total params
0.215     Total estimated model params size (MB)
INFO:lightning.

[{}]

# CNN1D_no with diffs

In [ ]:
str = 'cnn1d_no_diffs'

dm[str] = Custom_DataModule(xez = (d, e_d, z),
                       dataset_class=CNN1D_Dataset,
                       version='no', norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

c1 = Conv1dBlock(in_channels=1, out_channels=32,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv1dBlock(in_channels=32, out_channels=64,
                 kernel_size=2,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

flatten = nn.Flatten()

l1 = LinearBlock(in_features=128, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=144, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=144, out_features=1)

sequential = nn.Sequential(c1, c2, flatten,
                           l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 44.7 K
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
44.7 K    Trainable params
0         Non-trainable params
44.7 K    Total params
0.179     Total estimated model params size (MB)
INFO:lightning.

[{}]

# CNN1D_with with filters

In [ ]:
str = 'cnn1d_with_filters'

dm[str] = Custom_DataModule(xez = (f, e_f, z),
                       dataset_class=CNN1D_Dataset,
                       version='with', norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

c1 = Conv1dBlock(in_channels=1, out_channels=32,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv1dBlock(in_channels=32, out_channels=64,
                 kernel_size=2,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

flatten = nn.Flatten()

l1 = LinearBlock(in_features=448, out_features=336,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 336},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=336, out_features=336,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 336},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=336, out_features=1)

sequential = nn.Sequential(c1, c2, flatten,
                           l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 270 K 
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
270 K     Trainable params
0         Non-trainable params
270 K     Total params
1.081     Total estimated model params size (MB)
INFO:lightning.

[{}]

# CNN1D_with with diffs

In [ ]:
str = 'cnn1d_with_diffs'

dm[str] = Custom_DataModule(xez = (d, e_d, z),
                       dataset_class=CNN1D_Dataset,
                       version='with', norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

c1 = Conv1dBlock(in_channels=1, out_channels=32,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv1dBlock(in_channels=32, out_channels=64,
                 kernel_size=2,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

flatten = nn.Flatten()

l1 = LinearBlock(in_features=384, out_features=336,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 336},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=336, out_features=336,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 336},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=336, out_features=1)

sequential = nn.Sequential(c1, c2, flatten,
                           l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,max_epochs=250
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 248 K 
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
248 K     Trainable params
0         Non-trainable params
248 K     Total params
0.995     Total estimated model params size (MB)
INFO:lightning.

[{}]

# CNN1D_stack with filters

In [ ]:
str = 'cnn1d_stack_filters'

dm[str] = Custom_DataModule(xez = (f, e_f, z),
                       dataset_class=CNN1D_Dataset,
                       version='stack', norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

c1 = Conv1dBlock(in_channels=2, out_channels=32,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv1dBlock(in_channels=32, out_channels=64,
                 kernel_size=2,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

flatten = nn.Flatten()

l1 = LinearBlock(in_features=192, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=144, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=144, out_features=1)

sequential = nn.Sequential(c1, c2, flatten,
                           l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 54.0 K
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
54.0 K    Trainable params
0         Non-trainable params
54.0 K    Total params
0.216     Total estimated model params size (MB)
INFO:lightning.

[{}]

# CNN1D_stack with diffs

In [ ]:
str = 'cnn1d_stack_diffs'

dm[str] = Custom_DataModule(xez = (d, e_d, z),
                       dataset_class=CNN1D_Dataset,
                       version='stack', norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

c1 = Conv1dBlock(in_channels=2, out_channels=32,
                 kernel_size=4,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv1dBlock(in_channels=32, out_channels=64,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=MaxPool1d, pooling_layer_args=
                 {"kernel_size": 2})

flatten = nn.Flatten()

l1 = LinearBlock(in_features=128, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=144, out_features=144,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 144},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=144, out_features=1)

sequential = nn.Sequential(c1, c2, flatten,
                           l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 46.9 K
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
46.9 K    Trainable params
0         Non-trainable params
46.9 K    Total params
0.187     Total estimated model params size (MB)
INFO:lightning.

[{}]

# CNN2D with filters

In [ ]:
str = 'cnn2d_filters'

dm[str] = Custom_DataModule(xez = (f, e_f, z),
                       dataset_class=CNN2D_Dataset,
                       K=20, t_inf=10, t_sup=10, norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

c1 = Conv2dBlock(in_channels=1, out_channels=32,
                 kernel_size=4,
                 norm_layer=nn.BatchNorm2d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=nn.MaxPool2d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv2dBlock(in_channels=32, out_channels=64,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm2d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=nn.MaxPool2d, pooling_layer_args=
                 {"kernel_size": 2})

flatten = nn.Flatten()

l1 = LinearBlock(in_features=384, out_features=288,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 288},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=288, out_features=288,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 288},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=288, out_features=1)

sequential = nn.Sequential(c1, c2, flatten,
                           l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=175,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 214 K 
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
214 K     Trainable params
0         Non-trainable params
214 K     Total params
0.859     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module |

[{}]

# CNN2D with diffs

In [ ]:
str = 'cnn2d_diffs'

dm[str] = Custom_DataModule(xez = (d, e_d, z),
                       dataset_class=CNN2D_Dataset,
                       K=20, t_inf=10, t_sup=10, norm=True,
                       set_size=set_size,
                       batch_size=200, num_workers=1)

c1 = Conv2dBlock(in_channels=1, out_channels=32,
                 kernel_size=5,
                 norm_layer=nn.BatchNorm2d, norm_layer_args={'num_features': 32},
                 activation_function=nn.ReLU,
                 pooling_layer=nn.MaxPool2d, pooling_layer_args=
                 {"kernel_size": 2})

c2 = Conv2dBlock(in_channels=32, out_channels=64,
                 kernel_size=3,
                 norm_layer=nn.BatchNorm2d, norm_layer_args={'num_features': 64},
                 activation_function=nn.ReLU,
                 pooling_layer=nn.MaxPool2d, pooling_layer_args=
                 {"kernel_size": 2})

flatten = nn.Flatten()

l1 = LinearBlock(in_features=192, out_features=288,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 288},
                 activation_function=nn.ReLU)

l2 = LinearBlock(in_features=288, out_features=288,
                 norm_layer=nn.BatchNorm1d, norm_layer_args={'num_features': 288},
                 activation_function=nn.ReLU)

l3 = LinearBlock(in_features=288, out_features=1)

sequential = nn.Sequential(c1, c2, flatten,
                           l1, l2, l3)

m[str] = Reg(pytorch_module=sequential,
        loss_func=nn.MSELoss,
        optimizer=torch.optim.Adam, optimizer_args={"lr": 1e-3},
        tb_dir = str)

trainer = L.Trainer(max_epochs=250,
                        enable_progress_bar = False, log_every_n_steps=1,
                        accelerator="auto", devices=1,
                        logger = True)

trainer.fit(m[str], datamodule=dm[str])
trainer.test(m[str], datamodule=dm[str])

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: 
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module | Sequential | 159 K 
1 | loss_func      | MSELoss    | 0     
----------------------------------------------
159 K     Trainable params
0         Non-trainable params
159 K     Total params
0.639     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name           | Type       | Params
----------------------------------------------
0 | pytorch_module |

[{}]

In [ ]:
!zip -r ff_filters.zip ff_filters
#!zip -r ff_filters_errors.zip ff_filters_errors
#!zip -r ff_diffs.zip ff_diffs
#!zip -r ff_diffs_errors.zip ff_diffs_errors
#!zip -r cnn1d_no_filters.zip cnn1d_no_filters
#!zip -r cnn1d_no_diffs.zip cnn1d_no_diffs
#!zip -r cnn1d_with_filters.zip cnn1d_with_filters
#!zip -r cnn1d_with_diffs.zip cnn1d_with_diffs
#!zip -r cnn1d_stack_filters.zip cnn1d_stack_filters
#!zip -r cnn1d_stack_diffs.zip cnn1d_stack_diffs
!zip -r cnn2d_filters.zip cnn2d_filters
!zip -r cnn2d_diffs.zip cnn2d_diffs

  adding: ff_filters/ (stored 0%)
  adding: ff_filters/lightning_logs/ (stored 0%)
  adding: ff_filters/lightning_logs/version_0/ (stored 0%)
  adding: ff_filters/lightning_logs/version_0/squared_errors_stats (test)_std/ (stored 0%)
  adding: ff_filters/lightning_logs/version_0/squared_errors_stats (test)_std/events.out.tfevents.1712597390.605934e206d9.368.26 (deflated 10%)
  adding: ff_filters/lightning_logs/version_0/squared_errors_stats (train)_mean/ (stored 0%)
  adding: ff_filters/lightning_logs/version_0/squared_errors_stats (train)_mean/events.out.tfevents.1712597079.605934e206d9.368.17 (deflated 74%)
  adding: ff_filters/lightning_logs/version_0/squared_errors_stats (test)_size/ (stored 0%)
  adding: ff_filters/lightning_logs/version_0/squared_errors_stats (test)_size/events.out.tfevents.1712597390.605934e206d9.368.27 (deflated 10%)
  adding: ff_filters/lightning_logs/version_0/squared_errors_stats (validation)_median/ (stored 0%)
  adding: ff_filters/lightning_logs/version_0/s

In [ ]:
from google.colab import files

files.download("ff_filters.zip")
#files.download("ff_filters_errors.zip")
#files.download("ff_diffs.zip")
#files.download("ff_diffs_errors.zip")
#files.download("cnn1d_no_filters.zip")
#files.download("cnn1d_no_diffs.zip")
#files.download("cnn1d_with_filters.zip")
#files.download("cnn1d_with_diffs.zip")
#files.download("cnn1d_stack_filters.zip")
#files.download("cnn1d_stack_diffs.zip")
files.download("cnn2d_filters.zip")
files.download("cnn2d_diffs.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#!unzip ff_filters_errors.zip
!unzip ff_diffs.zip
!unzip ff_diffs_errors.zip
!unzip cnn1d_no_filters.zip
!unzip cnn1d_no_diffs.zip
!unzip cnn1d_with_filters.zip
!unzip cnn1d_with_diffs.zip
!unzip cnn1d_stack_filters.zip
!unzip cnn1d_stack_diffs.zip

Archive:  ff_diffs.zip
   creating: ff_diffs/
   creating: ff_diffs/lightning_logs/
   creating: ff_diffs/lightning_logs/version_0/
   creating: ff_diffs/lightning_logs/version_0/squared_errors_stats (train)_median/
  inflating: ff_diffs/lightning_logs/version_0/squared_errors_stats (train)_median/events.out.tfevents.1712571718.5987425793bc.2443.70  
   creating: ff_diffs/lightning_logs/version_0/losses_training_loss/
  inflating: ff_diffs/lightning_logs/version_0/losses_training_loss/events.out.tfevents.1712571718.5987425793bc.2443.67  
   creating: ff_diffs/lightning_logs/version_0/squared_errors_stats (train)_max/
  inflating: ff_diffs/lightning_logs/version_0/squared_errors_stats (train)_max/events.out.tfevents.1712571718.5987425793bc.2443.72  
   creating: ff_diffs/lightning_logs/version_0/squared_errors_stats (validation)_std/
  inflating: ff_diffs/lightning_logs/version_0/squared_errors_stats (validation)_std/events.out.tfevents.1712571717.5987425793bc.2443.65  
   creating: ff_